In [2]:
import math
#to read data from the web
import pandas_datareader as web 
#to perform numerical computations
import numpy as np 
#for data manipulation and analysis
import pandas as pd
#to normalize and scale the data
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
#to build the model layer by layer
from tensorflow.keras.models import Sequential
#for time series prediction - Long Short Term Memory
from tensorflow.keras.layers import Dense, LSTM
#to plot the graphs
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
#download market data from yahoo finance
import yfinance as yf